In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
os.listdir('./inputs/')

['feature_explain.xlsx',
 'sample_submit.txt',
 'test_a.xlsx',
 'test_b.xlsx',
 'train.xlsx']

In [29]:
train = pd.read_excel('inputs/train.xlsx')
test =  pd.read_excel('inputs/test_b.xlsx')
submission = pd.read_fwf('inputs/sample_submit.txt')
print(train.shape, test.shape, submission.shape)

(40000, 51) (12000, 50) (5, 2)


In [30]:
df = pd.read_excel('inputs/feature_explain.xlsx', sheet_name='train', skiprows=2)
index_num = 0
index_category = 0
for va in df.values:
    if va[1]=='Categorical':
        train=train.rename(columns={va[0]:va[1]+"_"+str(index_category)})
        test=test.rename(columns={va[0]:va[1]+"_"+str(index_category)})
        index_category+=1
    else:
        train=train.rename(columns={va[0]:va[1]+"_"+str(index_num)})
        test=test.rename(columns={va[0]:va[1]+"_"+str(index_num)})
        index_num+=1
        print(str(va[0])+"  "+str(index_num-1))
train.head()
# 把category和numerical 分开放
train=train[['CUST_UID','LABEL','Categorical_0','Categorical_1','Categorical_2','Categorical_3','Categorical_4','Categorical_5','Numerical_0','Numerical_1','Numerical_2','Numerical_3','Numerical_4','Numerical_5','Numerical_6','Numerical_7','Numerical_8','Numerical_9','Numerical_10','Numerical_11','Numerical_12','Numerical_13','Numerical_14','Numerical_15','Numerical_16','Numerical_17','Numerical_18','Numerical_19','Numerical_20','Numerical_21','Numerical_22','Numerical_23','Numerical_24','Numerical_25','Numerical_26','Numerical_27','Numerical_28','Numerical_29','Numerical_30','Numerical_31','Numerical_32','Numerical_33','Numerical_34','Numerical_35','Numerical_36','Numerical_37','Numerical_38','Numerical_39','Numerical_40','Numerical_41','Numerical_42']]
test=test[['CUST_UID','Categorical_0','Categorical_1','Categorical_2','Categorical_3','Categorical_4','Categorical_5','Numerical_0','Numerical_1','Numerical_2','Numerical_3','Numerical_4','Numerical_5','Numerical_6','Numerical_7','Numerical_8','Numerical_9','Numerical_10','Numerical_11','Numerical_12','Numerical_13','Numerical_14','Numerical_15','Numerical_16','Numerical_17','Numerical_18','Numerical_19','Numerical_20','Numerical_21','Numerical_22','Numerical_23','Numerical_24','Numerical_25','Numerical_26','Numerical_27','Numerical_28','Numerical_29','Numerical_30','Numerical_31','Numerical_32','Numerical_33','Numerical_34','Numerical_35','Numerical_36','Numerical_37','Numerical_38','Numerical_39','Numerical_40','Numerical_41','Numerical_42']]
features = train.columns.tolist()
for f in features:
    if train[f].isnull().sum()/train.shape[0]>0.3:
        train.drop(columns=[f],inplace=True)
        test.drop(columns=[f],inplace=True)

Column: AGN_CNT_RCT_12_MON  Description: Numerical
Column: ICO_CUR_MON_ACM_TRX_TM  Description: Numerical
Column: NB_RCT_3_MON_LGN_TMS_AGV  Description: Numerical
Column: AGN_CUR_YEAR_AMT  Description: Numerical
Column: AGN_CUR_YEAR_WAG_AMT  Description: Numerical
Column: AGN_AGR_LATEST_AGN_AMT  Description: Numerical
Column: ICO_CUR_MON_ACM_TRX_AMT  Description: Numerical
Column: COUNTER_CUR_YEAR_CNT_AMT  Description: Numerical
Column: PUB_TO_PRV_TRX_AMT_CUR_YEAR  Description: Numerical
Column: MON_12_EXT_SAM_TRSF_IN_AMT  Description: Numerical
Column: MON_12_EXT_SAM_TRSF_OUT_AMT  Description: Numerical
Column: MON_12_EXT_SAM_NM_TRSF_OUT_CNT  Description: Numerical
Column: MON_12_EXT_SAM_AMT  Description: Numerical
Column: CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT  Description: Numerical
Column: CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT  Description: Numerical
Column: MON_12_CUST_CNT_PTY_ID  Description: Categorical
Column: MON_12_TRX_AMT_MAX_AMT_PCTT  Description: Numerical
Column: CUR_YEAR_MON_AGV_T

In [35]:
#one hot 编码
df = pd.concat([train.assign(is_train = 1),test.assign(is_train = 0)])
df[['Categorical_1','Categorical_3']]=df[['Categorical_1','Categorical_3']].astype('object')
for f in df.columns.tolist():
    if df[f].dtype=='object':
        df[f].fillna("ety",inplace=True)
        df[f]=df[f].astype("object")
uid = df['CUST_UID']
df.drop(columns=['CUST_UID'],inplace=True)
df = pd.get_dummies(df)
df['CUST_UID'] = uid
train = df[df['is_train'] == 1]
test  = df[df['is_train'] == 0]
train.drop(columns=['is_train'],inplace=True)
test.drop(columns=['is_train'],inplace=True)

In [44]:
# # 字典编码
# def change_object_cols(se):
#     value = se.unique().tolist()
#     value.sort
#     return se.map(pd.Series(range(len(value)), index=value)).values

In [ ]:
# features = train.columns.tolist()[2:6]
# for f in features:
#     train[f] = change_object_cols(train[f].fillna(-1))
#     test[f] = change_object_cols(test[f].fillna(-1))

In [ ]:
# train.to_csv("preprocesses/train_dict.csv", index=False)
# test.to_csv("preprocesses/test_dict.csv", index=False)

In [ ]:
# train=pd.read_csv("preprocesses/train_dict.csv")
# test=pd.read_csv("preprocesses/test_dict.csv")

In [48]:
# 
features = []
for f in train.columns.tolist():
    if(train[f].dtype=='float64'):
        features.append(f)
print(features)
for f in features:
    median=train[f].median()
    train[f].fillna(median,inplace=True)
    test[f].fillna(median,inplace=True)

['LABEL', 'Numerical_1', 'Numerical_2', 'Numerical_6', 'Numerical_7', 'Numerical_8', 'Numerical_9', 'Numerical_10', 'Numerical_11', 'Numerical_12', 'Numerical_13', 'Numerical_14', 'Numerical_15', 'Numerical_16', 'Numerical_17', 'Numerical_18', 'Numerical_19', 'Numerical_20', 'Numerical_21', 'Numerical_23', 'Numerical_24', 'Numerical_25', 'Numerical_26', 'Numerical_27', 'Numerical_28', 'Numerical_29', 'Numerical_30', 'Numerical_31', 'Numerical_32', 'Numerical_33', 'Numerical_34', 'Numerical_35', 'Numerical_36', 'Numerical_37', 'Numerical_38', 'Numerical_39', 'Numerical_40', 'Numerical_41', 'Numerical_42']


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
train.to_csv("clean/train_dict_encode.csv", index=False)
test.to_csv("clean/test_dict_encode.csv", index=False)